### Model Training / Обучение модели
1. Разбиение выборки на тренировочную и тестовую
2. Стандартизация данных
3.
4.
5.
Split your dataset
Build model pipelines
Declare hyperparameters to tune
Fit and tune models with cross-validation
Evaluate metrics and select winner

In [ ]:
import sqlite3
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
# %matplotlib inline
import seaborn as sns
from sklearn.linear_model import ElasticNet, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split


In [ ]:
conn = sqlite3.connect('db\cian_362_150.db') 
          
df = pd.read_sql_query ('''
                               SELECT
                               flat_type, rooms, price, sale_type, mortgage, area,
                       living_area, kitchen_area, floor, floors, built_year, address, district, metro_station, seller,
                       housing_type, planning, ceiling_height, bathroom, balcony_loggia, repair, view,
                       finished_shell_condition, house_type, house_class, building_number, parking, elevators,
                       housing_line, floor_type, entrance_number, heating, unsafe_house, garbage_disposal, gas_supply,
                       description_text
                               FROM ad_data_cleaned
                               ''', conn)

1 Разбиение выборки на тренировочную и тестовую

In [ ]:
# создание выходной (целевой) переменной
y = df.tx_price
# создание входных переменных
X = df.drop('tx_price', axis = 1)

In [ ]:
# разбиение выбоки на тренировочную и тестовую в пропорции 80/20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [ ]:
# TODO сделать через assert как в книге
print(len(X_train), len(X_test), len(y_train), len(y_test))

2 Стандартизация данных

In [ ]:
# среднее и стреднеквадратическое отклонение тренировочного набора
train_mean = X_train.mean()
train_std = X_train.std()

In [ ]:
# стандартизация тренировочного набора
X_train_new = (X_train - train_mean) / train_std

In [ ]:
# описательная статистика для стандартизированных тренировочных данных
X_train_new.describe()